# pubmed 에서 논문의 제목과 초록을 크롤링하는 코드 입니다.

In [ ]:
import os
import time
from typing import List, Dict
from dotenv import load_dotenv
from Bio import Entrez
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# ---
## 1. 환경 변수 및 설정
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

DATABASE_PATH = "./chroma_pubmed_abstract_only"
COLLECTION_NAME = "rehabilitation_articles_openai"

# ---
## 2. ChromaDB 및 임베딩 설정
chroma_client = chromadb.PersistentClient(
    path=DATABASE_PATH,
    settings=Settings(allow_reset=True)
)

embedding_function = OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-3-large"
)

collection = chroma_client.get_or_create_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_function
)

# ---
## 3. PubMed 검색 설정
# Entrez API 사용 시 이메일은 필수입니다.
Entrez.email = "your_email@example.com"  

SEARCH_QUERY = (
    '(kinesiology OR "human movement" OR biomechanics OR "motor control" OR "physical activity") OR '
    '(pilates) OR (stretching OR flexibility OR "range of motion") OR '
    '("exercise therapy" OR "physical therapy" OR physiotherapy OR "rehabilitation exercise" OR "therapeutic exercise")'
)

# 현재 날짜 기준 최근 10년치 데이터
TODAY = time.strftime("%Y/%m/%d")
TEN_YEARS_AGO = f"{int(time.strftime('%Y')) - 10}/01/01"
DATE_RANGE = f"({TEN_YEARS_AGO}[PDAT] : {TODAY}[PDAT])"

# Entrez.esearch가 한 번에 반환할 수 있는 최대 ID 수 (PubMed의 제한)
MAX_IDS_PER_PAGINATION_REQUEST = 9999 
# Bio.Entrez.efetch 시 한 번에 처리할 논문 수 (ChromaDB 저장 배치와도 연관)
BATCH_SIZE = 500 

# ---
## 4. PubMed 검색 ID 가져오기 함수 (페이지네이션 구현)
def search_article_ids_paginated(query: str) -> List[str]:
    """
    Bio.Entrez.esearch를 사용하여 PubMed에서 논문 ID 목록을 검색합니다.
    10,000개 제한을 우회하기 위해 페이지네이션을 구현합니다.
    """
    full_query = f"{query} AND {DATE_RANGE}"
    print(f"PubMed에서 다음 쿼리로 전체 논문 수를 확인 중: '{full_query}'")
    
    all_id_list = []
    
    try:
        # 1. 전체 검색 결과 수 확인
        handle = Entrez.esearch(db="pubmed", term=full_query, retmax=0) # retmax=0으로 설정하여 개수만 가져옴
        results = Entrez.read(handle)
        handle.close()
        
        total_articles = int(results["Count"])
        print(f"🔍 해당 쿼리로 검색된 총 논문 수는 {total_articles}개 입니다.")
        
        # 2. 페이지네이션을 통해 ID 가져오기
        for start_index in range(0, total_articles, MAX_IDS_PER_PAGINATION_REQUEST):
            print(f"📄 {start_index}부터 {min(start_index + MAX_IDS_PER_PAGINATION_REQUEST, total_articles)}까지의 논문 ID 검색 중...")
            handle = Entrez.esearch(
                db="pubmed",
                term=full_query,
                retmax=MAX_IDS_PER_PAGINATION_REQUEST, # 한 번에 가져올 최대 ID 수
                retstart=start_index       # 검색 시작 위치 (오프셋)
            )
            results = Entrez.read(handle)
            handle.close()
            all_id_list.extend(results["IdList"])
            
            # NCBI 요청 제한을 준수하기 위해 잠시 대기
            time.sleep(0.5) 
            
            # 예상치 못한 상황으로 ID를 더 이상 가져오지 못하면 중단 (예: 검색 결과가 갑자기 줄거나)
            if not results["IdList"] or len(results["IdList"]) < MAX_IDS_PER_PAGINATION_REQUEST:
                # 마지막 페이지이거나 더 이상 결과가 없는 경우
                if start_index + len(results["IdList"]) >= total_articles:
                    break
            
    except Exception as e:
        print(f"🚨 논문 ID 검색 중 오류 발생: {e}")
        return []

    print(f"🔍 최종적으로 총 {len(all_id_list)}개의 논문 ID 검색됨.")
    return all_id_list

# ---
## 5. 초록 정보 가져오기 및 ChromaDB에 바로 저장 함수 (메모리 효율적)
def fetch_and_save_abstracts(id_list: List[str]):
    """
    주어진 ID 목록을 기반으로 PubMed에서 논문의 초록 정보를 가져와 ChromaDB에 바로 저장합니다.
    모든 초록을 메모리에 한꺼번에 로드하지 않으므로 메모리 효율적입니다.
    """
    if not id_list:
        print("❗ 저장할 논문 ID가 없습니다.")
        return

    print(f"총 {len(id_list)}개의 ID에서 초록을 가져와 ChromaDB에 저장 중...")
    
    temp_documents = []
    temp_metadatas = []
    temp_ids = []
    
    saved_count = 0 # 실제로 저장된 초록의 개수

    for start in range(0, len(id_list), BATCH_SIZE):
        end = start + BATCH_SIZE
        batch_ids = id_list[start:end]
        print(f"📚 배치 {start} ~ {min(end, len(id_list))} 처리 중...")
        try:
            handle = Entrez.efetch(db="pubmed", id=','.join(batch_ids), rettype="abstract", retmode="xml")
            records = Entrez.read(handle)
            handle.close()
            
            for article in records.get("PubmedArticle", []):
                pmid = article["MedlineCitation"]["PMID"]
                article_info = article["MedlineCitation"].get("Article", {})
                title = article_info.get("ArticleTitle", "")
                
                abstract_parts = article_info.get("Abstract", {}).get("AbstractText", [])
                abstract = " ".join(abstract_parts)
                
                if abstract: 
                    temp_documents.append(abstract)
                    temp_metadatas.append({"title": title})
                    temp_ids.append(str(pmid))
            
            # 일정량의 데이터가 모이면 ChromaDB에 저장
            if len(temp_documents) >= BATCH_SIZE: 
                print(f"✅ ChromaDB에 중간 저장: {len(temp_documents)}개 항목 추가 중...")
                collection.add(
                    documents=temp_documents,
                    metadatas=temp_metadatas,
                    ids=temp_ids
                )
                saved_count += len(temp_documents)
                temp_documents.clear()
                temp_metadatas.clear()
                temp_ids.clear()
            
            time.sleep(0.5) # NCBI 요청 제한 준수
            
        except Exception as e:
            print(f"❌ 배치 {start}-{end} 가져오기 오류 발생:", e)
            time.sleep(1) 
            continue 
            
    # 루프 종료 후 남아있는 항목들 최종 저장
    if temp_documents:
        print(f"✅ ChromaDB에 최종 저장: 남은 {len(temp_documents)}개 항목 추가 중...")
        collection.add(
            documents=temp_documents,
            metadatas=temp_metadatas,
            ids=temp_ids
        )
        saved_count += len(temp_documents)
    
    print(f"📄 총 {saved_count}개의 유효한 초록이 ChromaDB에 저장되었습니다.")

# ---
## 6. 메인 실행 블록
if __name__ == "__main__":
    print(f"'{COLLECTION_NAME}' 컬렉션에 접근했습니다. (OpenAI `text-embedding-3-large` 사용)")
    try:
        # 페이지네이션 로직을 사용하는 ID 검색 함수 호출
        id_list = search_article_ids_paginated(SEARCH_QUERY)
        
        # 검색된 ID가 없으면 종료
        if not id_list:
            print("❗ 논문 ID를 찾지 못했습니다. 작업을 종료합니다.")
            exit()
        
        # 검색된 ID로 초록 정보 가져오기 및 ChromaDB에 바로 저장
        fetch_and_save_abstracts(id_list)
        
        print("🎉 완료: 모든 초록 데이터 수집 및 ChromaDB에 저장이 완료되었습니다.")

    except Exception as e:
        print(f"🚨 전체 실행 중 오류 발생: {e}")

'rehabilitation_articles_openai' 컬렉션에 접근했습니다. (OpenAI `text-embedding-3-large` 사용)
EDirect를 사용하여 PubMed에서 다음 쿼리로 논문 ID를 검색 중: '(kinesiology OR "human movement" OR biomechanics OR "motor control" OR "physical activity") OR (pilates) OR (stretching OR flexibility OR "range of motion") OR ("exercise therapy" OR "physical therapy" OR physiotherapy OR "rehabilitation exercise" OR "therapeutic exercise") AND (2015/01/01[PDAT] : 2025/07/21[PDAT])'
🚨 EDirect가 시스템 경로에 없거나 설치되지 않았습니다. WSL 환경에 EDirect를 먼저 설치해주세요.
❗ 논문 ID를 찾지 못했습니다. 프로그램을 종료합니다.
❗ 저장할 논문 ID가 없습니다.
🎉 완료: 모든 초록 데이터 수집 및 ChromaDB에 저장이 완료되었습니다.


: 